# [esasscookbook](https://erosita.mpe.mpg.de/edr/DataAnalysis/esasscookbook.html)

In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pylab import cm

import matplotlib.dates as mdates
from matplotlib.pyplot import MultipleLocator
import matplotlib.dates as mdates
from matplotlib.transforms import Transform
from matplotlib.ticker import (AutoLocator, AutoMinorLocator)
from matplotlib.dates import DateFormatter
#import matplotlib.cm as cm
import matplotlib.colors as colors
from collections import OrderedDict


# Import required packages
import numpy as np
import pandas as pd
#import plotly_express as px


from scipy.stats import spearmanr#
from scipy.stats.stats import pearsonr
from astropy import constants
from astropy import units as u
from astropy import units

from datetime import datetime, date, time, timezone
from datetime import datetime
from datetime import timedelta
from astropy.time import Time
from astropy.io import ascii




%matplotlib inline
%config InlineBackend.figure_format='svg'


import matplotlib.font_manager as fm
# Collect all the font names available to matplotlib
font_names = [f.name for f in fm.fontManager.ttflist]

# Edit the font, font size, and axes width
mpl.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

In [6]:
import pandas as pd
import numpy as np
import os

def get_obsids(path):
    dirname=os.listdir(path)
    obsids=[]
    for i in dirname:
        if i.isdigit():
            obsids.append(i)
    obsids.sort()        
    return obsids

def drop_index(data):
    data=data.reset_index(drop=True)
    return data

def get_info(data,label,label_err=None):
    return min(data[label]),max(data[label]),np.mean(data[label])
    

In [7]:
from astropy.time import Time
from astropy.io import fits
import time
from matplotlib.pyplot import MultipleLocator
import matplotlib.dates as mdates

def datetime2mjd(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    x=mdates.date2num(x)
    y = x - mjd_minus_mdates_num   
    return y

def mjd2datetime(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y= x + mjd_minus_mdates_num
    y= mdates.num2date(y)
    return y



def datenums2mjd(x):
    #x=mdates.date2num(x)
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    y = x - mjd_minus_mdates_num   
    return y

def mjd2numsdate(x):
    mjd_ref=59000
    mjd_minus_mdates_num=mdates.date2num(convert_xaxis_time(mjd_ref))-mjd_ref
    
    y= x + mjd_minus_mdates_num
    #y= mdates.num2date(y)
    return y


def convert_xaxis_mjd(time):
    return Time(time).mjd   

def convert_xaxis_time(mjd):
    return Time(mjd,format='mjd').to_datetime()


def date2yday(x):
    """
    x is in matplotlib datenums, so they are floats.
    """
    y = x - mdates.date2num(datetime(2018, 1, 1))
    return y

def yday2date(x):
    """
    return a matplotlib datenum (x is days since start of year of 2018)
    """
    y = x + mdates.date2num(datetime(2018, 1, 1))
    return y


def convert_partial_year(numbers):
    datetimes=[]
    for number in numbers:
        year = int(number)
        d = timedelta(days=(number - year)*(365 + is_leap(year)))
        day_one = datetime(year,1,1)
        date = d + day_one
        datetimes.append(date)
    return datetimes


def is_leap(year):
    if not year%4 and  year%100 or not year%400:
        return True
    return False


def convert_mjd(times):
    timesmjd=[]
    for i in times:
        timesmjd.append(Time(i).mjd)
    return timesmjd  


def convert_date(times):
    timesdate=[]
    for i in times:
        timesdate.append(Time(i,format='mjd').datetime)
    return timesdate

def convert_date_single(time):
    timedate=Time(time,format='mjd').datetime
    return timedate

In [8]:
from adjustText import adjust_text
import matplotlib as mpl
def set_ax_tick(ax):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, direction='in', top='on',)
    ax.xaxis.set_tick_params(which='minor', size=5, width=2, direction='in', top='on')
    ax.yaxis.set_tick_params(which='major', size=10, width=2, direction='in', right='on')
    ax.yaxis.set_tick_params(which='minor', size=5, width=2, direction='in', right='on')

def set_ax_locator(ax,xma=1,xmi=0.2,yma=1,ymi=0.2):
    ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(xma))
    ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(xmi))
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(yma))
    ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(ymi))
    
def plot_secax(ax,mi_interval=365,ma_interval=365*2,rotation=30,):
    secax1 = ax.secondary_xaxis('top', functions=(mjd2numsdate,datenums2mjd))
    secax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    secax1.xaxis.set_major_locator(mdates.DayLocator(interval=ma_interval))
    secax1.xaxis.set_minor_locator(mdates.DayLocator(interval=mi_interval))
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='minor', size=5, width=2, direction='out')
    secax1.xaxis.set_tick_params(which='major', size=10, width=2, direction='out', rotation=rotation,)    
    
def set_ax_legend(ax,bbox_to_anchor=(0.01, 0.99)):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=0,)
    handles, labels = ax.get_legend_handles_labels()
    # remove the errorbars
    #hdl = [h[0] for h in handles]
    hdl = handles
    labels_dict=dict(zip(labels, hdl)) #key,values
    by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
    #by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=bbox_to_anchor,
              loc=2, numpoints=1,ncol=1,fontsize=11.)
    
def set_ax_legend_sequence(ax,bbox_to_anchor=(0.01, 0.99)):
    ax.xaxis.set_tick_params(which='major', size=10, width=2, rotation=0,)
    handles, labels = ax.get_legend_handles_labels()
    # remove the errorbars
    #hdl = [h[0] for h in handles]
    hdl = handles
    labels_dict=dict(zip(labels, hdl)) #key,values
    #by_label=OrderedDict(sorted(labels_dict.items(),key=lambda t:t[0]))
    by_label = OrderedDict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=bbox_to_anchor,
              loc=2, numpoints=1,ncol=1,fontsize=11.)    
    

def set_mag_ylim(ax):
    bottom, top = ax.set_ylim()
    if bottom< top:
        ax.set_ylim(top,bottom)   
        
        
def set_mag_xlim(ax):
    bottom, top = ax.set_xlim()
    if bottom< top:
        ax.set_xlim(top,bottom)         

## eROSITA/eFEDS main catalogue

In [21]:
eFEDS_c001_main=fits.open('/Users/lyubing/Downloads/erosita/eFEDS_c001_main_V7.4.fits')

In [22]:
eFEDS_c001_main.info()

Filename: /Users/lyubing/Downloads/erosita/eFEDS_c001_main_V7.4.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    662   27910R x 162C   [22A, J, J, D, D, E, 1D, 1D, 1D, E, E, E, E, E, E, E, E, E, E, E, E, 1L, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1J, 1J, 1J, 1J, 1J, 1J, 1J, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D, 1D]   


In [23]:
eFEDS_c001_maindata=eFEDS_c001_main[1].data

In [29]:
eFEDS_c001_maindata.columns

ColDefs(
    name = 'Name'; format = '22A'
    name = 'ID_SRC'; format = 'J'
    name = 'ID_hard'; format = 'J'
    name = 'RA'; format = 'D'; unit = 'deg'
    name = 'DEC'; format = 'D'; unit = 'deg'
    name = 'RADEC_ERR'; format = 'E'; unit = 'arcsec'
    name = 'RA_CORR'; format = '1D'; unit = 'deg'
    name = 'DEC_CORR'; format = '1D'; unit = 'deg'
    name = 'RADEC_ERR_CORR'; format = '1D'; unit = 'arcsec'
    name = 'EXT'; format = 'E'; unit = 'arcsec'
    name = 'EXT_ERR'; format = 'E'; unit = 'arcsec'
    name = 'EXT_LIKE'; format = 'E'
    name = 'DET_LIKE'; format = 'E'
    name = 'ML_RATE'; format = 'E'; unit = 'counts/s'
    name = 'ML_RATE_ERR'; format = 'E'; unit = 'counts/s'
    name = 'ML_CTS'; format = 'E'; unit = 'counts'
    name = 'ML_CTS_ERR'; format = 'E'; unit = 'counts'
    name = 'ML_FLUX'; format = 'E'; unit = 'erg/cm^2/s'
    name = 'ML_FLUX_ERR'; format = 'E'; unit = 'erg/cm^2/s'
    name = 'ML_EXP'; format = 'E'; unit = 'seconds'
    name = 'ML_BKG'; forma

## eROSITA/eFEDS variable point sources

In [24]:
eFEDS_c001_Lightcurves_softhard_V=fits.open('/Users/lyubing/Downloads/erosita/eFEDS_c001_Lightcurves_softhard_V5.fits')

In [25]:
eFEDS_c001_Lightcurves_softhard_V.info()

Filename: /Users/lyubing/Downloads/erosita/eFEDS_c001_Lightcurves_softhard_V5.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (19633,)   uint8   
  1  data_all_eFEDS_variability.tsv    1 BinTableHDU    175   27910R x 62C   [I, E, D, D, D, D, D, L, D, D, L, E, L, E, E, L, E, E, E, E, D, D, D, D, D, L, D, D, L, E, L, E, E, L, E, E, E, E, D, D, D, D, D, L, D, D, L, E, L, E, E, L, E, E, E, L, L, L, L, L, L, L]   


In [26]:
eFEDS_c001_Lightcurves_softhard_Vdata=eFEDS_c001_Lightcurves_softhard_V[1].data

In [27]:
eFEDS_c001_Lightcurves_softhard_V[0].header

SIMPLE  =                    T / Standard FITS format                           
BITPIX  =                    8 / Character data                                 
NAXIS   =                    1 / Text string                                    
NAXIS1  =                19633 / Number of characters                           
VOTMETA =                    T / Table metadata in VOTable format               
EXTEND  =                    T / There are standard extensions                  
COMMENT                                                                         
COMMENT The data in this primary HDU consists of bytes which                    
COMMENT comprise a VOTABLE document.                                            
COMMENT The VOTable describes the metadata of the table contained               
COMMENT in the following BINTABLE extension.                                    
COMMENT Such a BINTABLE extension can be used on its own as a perfectly         
COMMENT good table, but the 

In [32]:
len(eFEDS_c001_Lightcurves_softhard_Vdata['SRCID'])#.columns

27910

In [30]:
eFEDS_c001_Lightcurves_softhard_Vdata['SRCID']#.columns

array([    1,     2,     3, ..., 32680, 32681, 32683], dtype=int16)

In [14]:
len(eFEDS_c001_Lightcurves_softhard_Vdata)

27910

# 3C390.3

c1= fits.open('/Users/lyubing/heasoft/eROSITA_install/eSASS/data/3C390/pi00_900060_020_EventList_c001.fits')



c1.info()

In [15]:
ls /Users/lyubing/Downloads/erosita/3C390/*.fits

/Users/lyubing/Downloads/erosita/3C390/pi00_900060_020_EventList_c001.fits
/Users/lyubing/Downloads/erosita/3C390/pi00_900068_020_EventList_c001.fits
/Users/lyubing/Downloads/erosita/3C390/pi00_900069_020_EventList_c001.fits
/Users/lyubing/Downloads/erosita/3C390/pi00_900070_020_EventList_c001.fits


## evtool 

In [ ]:
evtool eventfiles="pi00_900060_020_EventList_c001.fits pi00_900060_020_EventList_c001.fits pi00_900069_020_EventList_c001.fits pi00_900070_020_EventList_c001.fits" outfile="events_image_comb.fits" image=yes emin=0.5 emax=2.0


#keV

## expmap

In [ ]:
expmap inputdatasets="events_image_comb.fits" emin=0.5 emax=2.0 \
          templateimage="events_image_comb.fits" mergedmaps="output_expmap.fits" 

#keV

## ermask

In [ ]:
ermask expimage="output_expmap.fits" detmask="detmask.fits"

## erbox

In [ ]:
erbox images="events_image_comb.fits" boxlist="boxlist_local.fits" emin=500 emax=2000 expimages="output_expmap.fits" detmasks="detmask.fits" bkgima_flag=N ecf=1

#eV


## erbackmap 

In [ ]:
erbackmap image="events_image_comb.fits" expimage="output_expmap.fits" boxlist="boxlist_local.fits" detmask="detmask.fits" bkgimage="bkg_map.fits" emin=500 emax=2000 cheesemask="cheesemask.fits"



## erbox

In [ ]:
erbox images="events_image_comb.fits" boxlist="boxlist_map.fits" expimages="output_expmap.fits" detmasks="detmask.fits" bkgimages="bkg_map.fits" emin=500 emax=2000 bkgima_flag=Y ecf=1


## ermldet

In [ ]:
ermldet mllist="mllist.fits" boxlist="boxlist_map.fits" images="events_image_comb.fits" expimages="output_expmap.fits" detmasks="detmask.fits" bkgimages="bkg_map.fits" extentmodel=beta srcimages="sourceimage.fits" emin=500 emax=2000


In [ ]:
ermldet mllist="mllist.fits" boxlist="boxlist_map.fits" images="events_image_comb.fits" expimages="output_expmap.fits" detmasks="detmask.fits" bkgimages="bkg_map.fits" extentmodel=gaussian srcimages="sourceimage.fits" emin=500 emax=2000


Note that the extended model can be either Gaussian or Beta, depending on the user necessities.

## catprep

In [ ]:
catprep infile="mllist.fits" outfile="catalog.fits"


## srctool

In [ ]:
srctool eventfiles="events_image_comb.fits" todo=ALL srccoord="catalog.fits" srcreg='fk5;circle * * 60"' backreg='fk5;annulus * * 90" 120"' clobber=yes

In the following, a step-by-step example of how to obtain source catalogues and products is shown. The example uses the default values for each eSASS task, and not all the task options are shown. For more specific requirements refer to the individual task pages.

1. evtool merges the input event files and creates an image in a given energy band:

 $ evtool eventfiles="events1.fits events2.fits ... events7.fits" \
	  outfile="events_image_comb.fits" image=yes emin=0.5 emax=2.0
The output file events_image_comb.fits will have events and image extensions.

2. expmap computes the exposure map in a given energy band:

 $ expmap inputdatasets="events_image_comb.fits" emin=0.5 emax=2.0 \
          templateimage="events_image_comb.fits" mergedmaps="output_expmap.fits" 
3. ermask creates a detection mask for the eSASS source detection chain:

 $ ermask expimage="output_expmap.fits" detmask="detmask.fits"
4. erbox in local mode is the first step on the eSASS source detection chain.

 $ erbox images="events_image_comb.fits" boxlist="boxlist_local.fits" emin=500 emax=2000 \
	 expimages="output_expmap.fits" detmasks="detmask.fits" bkgima_flag=N ecf=1
Note that bkgima_flag=N since the background map is created in the next step. An energy conversion factor (ecf) is necessary for erbox to run.

5. erbackmap creates a background map by masking sources detected by erbox:

 $ erbackmap image="events_image_comb.fits" expimage="output_expmap.fits" \
	     boxlist="boxlist_local.fits" detmask="detmask.fits" bkgimage="bkg_map.fits" emin=500 \
	     emax=2000 cheesemask="cheesemask.fits"
6. erbox in map mode is the second step on the eSASS source detection chain:

 $ erbox images="events_image_comb.fits" boxlist="boxlist_map.fits" expimages="output_expmap.fits" \
         detmasks="detmask.fits" bkgimages="bkg_map.fits" emin=500 emax=2000 ecf=1
7. ermldet characterise the detected sources and determines source parameters:

 $ ermldet mllist="mllist.fits" boxlist="boxlist_map.fits" images="events_image_comb.fits" \
	   expimages="output_expmap.fits" detmasks="detmask.fits" bkgimages="bkg_map.fits" \
	   extentmodel=beta srcimages="sourceimage.fits" emin=500 emax=2000
Note that the extended model can be either Gaussian or Beta, depending on the user necessities.

8. catprep converts the output of ermldet into the catalog file format:

 $ catprep infile="mllist.fits" outfile="catalog.fits"
9. srctool derives source products (spectra, light-curves and corresponding instrumental files):

 $ srctool eventfiles="events_image_comb.fits" todo=ALL srccoord="catalog.fits" \
           srcreg='fk5;circle * * 60"' backreg='fk5;annulus * * 90" 120"' clobber=yes
In this case, srctool will extract all types of source and background products for all detected sources in the catalog.fits file.

2 Quick analysis guide

2.7 How to create an event file from a given region?
evtool can filter an input event list according to a specified region (file or expression):

 $ evtool eventfiles="events.fits" outfile="events_image_comb.fits" region="fk5;circle(90,23,1.2)"
or

 $ evtool eventfiles="events.fits" outfile="events_image_comb.fits" region="regionfile.reg"
The region files must be created before calling evtool with your favourite text editor, and it must contain one region per line.

2.8 How to extract an event file using good time intervals (GTIs)?
evtool can filter an input event list according to a specified time range (STDGTI extension or time range):

 $ evtool eventfiles="events.fits" gti="TSTART TSTOP" outfile="events_filtered.fits"
where TSTART is the beginning of the GTI and TSTOP is the end of the GTI in seconds. TSTART and TSTOP values must be covered by the used observation.

2.9 Which good time intervals (GTIs) should I use to produce an exposure map?
The option gtitype in the task expmap specifies the type of good time intervals (GTIs) to be used for exposure map creation. The default value is GTI. If the user wishes to also exclude flared times, obtained e.g. with the task flaregti, then gtitype=FLAREGTI should be used:

 $ expmap eventfiles="events.fits" gtitype=FLAREGTI outfile="events_flarefiltered.fits"
2.10 How to extract a light-curve of a point source?
One of the source products from srctool is a light curve ( todo="LC LCCORR" ) option), for example:

 $ srctool eventfiles="events.fits" exttype="POINT" srccoord="fk5;99.0,-20.0" todo="LC LCCORR"
The option srccoord should include the coordinates of the source of interest within the observation. One can choose the instrument (insts option), the binning (lcpars option), energy range (lcemin and lcemax options), etc.

2.11 How to extract a spectrum/ARF/RMF for a point source?
srctool can extract the spectrum of a point source (exttype="POINT"):

 $ srctool eventfiles="events.fits" exttype="POINT" srccoord="fk5;99.0,-20.0" todo="SPEC ARF RMF"
2.12 How to extract a spectrum/ARF/RMF for an extended source?
srctool can extract the spectrum of an extended source (exttype="tophat"):

 $ srctool eventfiles="events.fits" exttype="tophat" srccoord="fk5;20.0,-99.0" \
	   todo="SPEC ARF RMF" PSFtype=none extpars=60
The extpars option depends on the value of the exttype parameter. When using a map to describe a source extent, the PSF corrections must be turned off (PSFtype=none). However, when extracting source products for relatively compact and circularly symmetric extended source, one should turn on PSF modelling, since PSF losses are likely to be important.